In [5]:
import pandas as pd

# Load the dataset
file_path = 'dementia_dataset.csv'
dementia_data = pd.read_csv(file_path)

# Display the first few rows of the dataset for inspection
dementia_data.head()

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,OAS2_0002,OAS2_0002_MR1,Demented,1,0,M,R,75,12,NaN,23.0,0.5,1678,0.736,1.046
3,OAS2_0002,OAS2_0002_MR2,Demented,2,560,M,R,76,12,NaN,28.0,0.5,1738,0.713,1.010
4,OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NaN,22.0,0.5,1698,0.701,1.034


In [6]:
# Data Preparation Steps

# 1. Handling Missing Data
# Checking for missing values in SES, MMSE, and nWBV
missing_values = dementia_data[['SES', 'MMSE', 'nWBV']].isnull().sum()
missing_values


SES     19
MMSE     2
nWBV     0
dtype: int64

In [7]:
# 2. Data Cleaning
# Checking the data types and unique values in SES
ses_types = dementia_data['SES'].dtype
ses_types


dtype('float64')

In [8]:
unique_ses = dementia_data['SES'].unique()
unique_ses

array([ 2., nan,  3.,  4.,  1.,  5.])

In [11]:
# Imputation Strategy: Using median SES values for groups defined by similar education levels (EDUC)
# First, calculate the median SES for each EDUC level
median_ses_per_educ = dementia_data.groupby('EDUC')['SES'].median()
median_ses_per_educ


EDUC
6     4.0
8     5.0
11    4.0
12    3.0
13    2.0
14    3.0
15    2.0
16    2.0
17    1.0
18    2.0
20    1.0
23    1.0
Name: SES, dtype: float64

In [12]:

# Function to impute SES based on EDUC
def impute_ses(row):
    if pd.isna(row['SES']) and not pd.isna(row['EDUC']):
        return median_ses_per_educ[row['EDUC']]
    return row['SES']

# Applying the imputation function
dementia_data['SES'] = dementia_data.apply(impute_ses, axis=1)

# Checking if there are any remaining missing SES values
remaining_missing_ses = dementia_data['SES'].isnull().sum()
remaining_missing_ses


0

In [14]:
# Structuring the Data for Analysis

# Preparing data for MMSE scores analysis by SES
mmse_ses_data = dementia_data[['Subject ID', 'SES', 'Visit', 'MMSE']].copy()
mmse_ses_data.dropna(subset=['MMSE'], inplace=True)  # Dropping rows where MMSE is missing

# Preparing data for nWBV analysis by SES
nwbv_ses_data = dementia_data[['Subject ID', 'SES', 'Visit', 'nWBV']].copy()

# Data is now structured to compare MMSE and nWBV across different SES levels
# Displaying the first few rows of the prepared data for MMSE and nWBV
mmse_ses_data.head(), nwbv_ses_data.head()


(  Subject ID  SES  Visit  MMSE
 0  OAS2_0001  2.0      1  27.0
 1  OAS2_0001  2.0      2  30.0
 2  OAS2_0002  3.0      1  23.0
 3  OAS2_0002  3.0      2  28.0
 4  OAS2_0002  3.0      3  22.0,
   Subject ID  SES  Visit   nWBV
 0  OAS2_0001  2.0      1  0.696
 1  OAS2_0001  2.0      2  0.681
 2  OAS2_0002  3.0      1  0.736
 3  OAS2_0002  3.0      2  0.713
 4  OAS2_0002  3.0      3  0.701)

In [15]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Linear Mixed-Effects Model for MMSE Scores
# Model: MMSE ~ Visit + SES + Visit:SES (interaction term), random intercepts for subjects

# Fitting the model
lme_mmse_model = smf.mixedlm("MMSE ~ Visit * SES", mmse_ses_data, groups=mmse_ses_data["Subject ID"], 
                             re_formula="~Visit").fit()

# Displaying the model summary
lme_mmse_model.summary()


<class 'statsmodels.iolib.summary2.Summary'>
"""
           Mixed Linear Model Regression Results
============================================================
Model:               MixedLM  Dependent Variable:  MMSE     
No. Observations:    371      Method:              REML     
No. Groups:          150      Scale:               2.1844   
Min. group size:     1        Log-Likelihood:      -861.4456
Max. group size:     5        Converged:           Yes      
Mean group size:     2.5                                    
------------------------------------------------------------
                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------
Intercept         29.188    0.573 50.905 0.000 28.064 30.312
Visit             -0.243    0.322 -0.756 0.450 -0.873  0.387
SES               -0.406    0.213 -1.906 0.057 -0.823  0.011
Visit:SES         -0.137    0.121 -1.135 0.256 -0.373  0.099
Group Var          2.244    0.993                           
Group x Visit Cov  1.422    0.253                           
Visit Var          0.901    0.194                           
============================================================

"""

In [17]:
# Re-examining the nWBV dataset for inconsistencies or missing data
nwbv_ses_data_check = dementia_data[['Subject ID', 'SES', 'Visit', 'nWBV']].copy()

# Checking for missing values
nwbv_missing_values = nwbv_ses_data_check.isnull().sum()

# Checking the data types
nwbv_data_types = nwbv_ses_data_check.dtypes

nwbv_missing_values, nwbv_data_types



(Subject ID    0
 SES           0
 Visit         0
 nWBV          0
 dtype: int64,
 Subject ID     object
 SES           float64
 Visit           int64
 nWBV          float64
 dtype: object)